
# LangGraph Tool Calling - Simple Explanation

This code builds a **conversational AI system** that can use **tools/functions** to answer questions, using LangGraph + Databricks LLM.


## 🛠️ **Tool Definition**
```python
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b
```
A simple function that multiplies two numbers. The LLM can call this when users ask math questions.

## 🤖 **LLM Setup**
```python
llm = ChatDatabricks(endpoint="databricks-gpt-oss-120b")
llm_with_tools = llm.bind_tools([multiply])  # Give LLM access to the multiply tool
```

## 🕸️ **Graph Structure**


START → tool_calling_llm → [conditional] → tools → tool_calling_llm → END




- **tool_calling_llm**: LLM decides whether to use tools or respond directly
- **tools**: Executes the actual tool (multiply function)
- **Conditional edge**: Routes to tools only if LLM wants to use them

## 🔄 **How It Works**
1. User asks: *"What is the weather in Tokyo? and what is 2 * 3?"*
2. LLM analyzes the question
3. For "2 * 3", it calls the `multiply(2, 3)` tool
4. Gets result `6` from the tool
5. Responds with both weather info and math result

## 🎯 **Key Features**
- **Smart routing**: LLM automatically decides when to use tools
- **Tool integration**: Functions become available to the AI
- **Conversation flow**: Maintains chat history and context
- **Conditional logic**: Uses tools only when needed

The output shows each message in the conversation, including tool calls and responses.

In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


from langgraph.graph import MessagesState

# load_dotenv()


import mlflow
mlflow.langchain.autolog()

class State(MessagesState):
    pass



def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    """Example:
    >>> multiply(2, 3)
    6
    """
    """Args:
    a: int
    b: int
    """
    """Returns:
    int
    """
    return a * b


llm = ChatDatabricks(endpoint = "databricks-gpt-oss-120b")

llm_with_tools = llm.bind_tools([multiply])

def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)


builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)

graph = builder.compile()

result = graph.invoke({"messages": [HumanMessage("Hello,what is the weather in Tokyo? and what is 2 * 3?")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")




In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


from langgraph.graph import MessagesState

# load_dotenv()


import mlflow
mlflow.langchain.autolog()

class State(MessagesState):
    pass



def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    """Example:
    >>> multiply(2, 3)
    6
    """
    """Args:
    a: int
    b: int
    """
    """Returns:
    int
    """
    return a * b


llm = ChatDatabricks(endpoint = "databricks-gpt-oss-120b")

llm_with_tools = llm.bind_tools([multiply])

def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply]))


builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

result = graph.invoke({"messages": [HumanMessage("Hello,what is the weather in Tokyo? and what is 2 * 3?")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")




LangChain Inbuilt tools https://python.langchain.com/docs/integrations/tools/

In [0]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


from langgraph.graph import MessagesState

# load_dotenv()


import mlflow
mlflow.langchain.autolog()

class State(MessagesState):
    pass




llm = ChatDatabricks(endpoint = "databricks-gpt-oss-120b")

llm_with_tools = llm.bind_tools([wikipedia])

def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([wikipedia]))


builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

result = graph.invoke({"messages": [HumanMessage("Tell me about databricks.")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")


